# 融资利率套利

In [2]:
import os
import sys
import datetime as dt

import ccxt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")

sys.path.insert(0, "/home/scofieldchen0011/quant-research")

from scripts.fetch_exchange_ohlcv import get_ohlcv
from scripts.fetch_binance_funding_rates import fetch_funding_rates

In [15]:
# 融资套利的货币对
symbol = "FTM/USDT"
# 现货交易所
spot_exchange_id = "binance"
# 合约交易所
perp_exchange_id = "binanceusdm"
# K线时间周期
timeframe = "4h"
# 开始日期
start_date = dt.datetime(2020, 1, 1)
# 结束日期
end_date = dt.datetime(2024, 10, 30)
# 存储数据的文件夹
data_dir = "/home/scofieldchen0011/quant-research/data/funding_arbitrage"

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [16]:
# 获取现货历史价格
spot_exchange = getattr(ccxt, spot_exchange_id)()
spot_ohlcv = get_ohlcv(spot_exchange, symbol, timeframe, start_date, end_date)
print(f"Downloaded spot data for {symbol}")

# 获取永续合约历史价格
perp_exchange = getattr(ccxt, perp_exchange_id)()
perp_ohlcv = get_ohlcv(perp_exchange, symbol, timeframe, start_date, end_date)
print(f"Downloaded perpetual data for {symbol}")

# 获取融资利率历史数据
funding_rates = fetch_funding_rates(symbol.replace("/", ""), start_date, end_date)
print(f"Downloaded funding rates for {symbol}")

Downloaded spot data for FTM/USDT
Downloaded perpetual data for FTM/USDT
Downloaded funding rates for FTM/USDT


In [17]:
# 将数据存储到本地csv，方便后续分析
base_token = symbol.split("/")[0].lower()
spot_ohlcv.to_csv(
    f"{data_dir}/{spot_exchange_id}_{base_token}_{timeframe}.csv", index=True
)
perp_ohlcv.to_csv(
    f"{data_dir}/{perp_exchange_id}_{base_token}_{timeframe}.csv", index=True
)
funding_rates.to_csv(f"{data_dir}/funding_rates_{base_token}.csv", index=True)

In [41]:
# 数据预处理

# 合并现货和合约价格，仅保留开盘价和收盘价
df_joined = pd.merge(
    spot_ohlcv[["open", "close"]],
    perp_ohlcv[["open", "close"]],
    on="timestamp",
    suffixes=("_spot", "_perp"),
)

# 合并融资利率
# 融资利率的时间戳精确到毫秒且有时不为零，先将时间戳转化为妙
funding_rates.index.name = "timestamp"
funding_rates.index = funding_rates.index.floor("s")
df_joined = pd.merge(
    df_joined, funding_rates["funding_rate"], on="timestamp", how="left"
)

# 融资利率每8小时计算一次，缺失值用0填充
df_joined["funding_rate"] = df_joined["funding_rate"].fillna(0)

In [52]:
# 计算累计融资利率
cumulative_days = 3
df_joined["cumulative_funding"] = (
    df_joined["funding_rate"].rolling(dt.timedelta(days=cumulative_days)).sum()
)